In [1]:
import numpy as np
import librosa
from scipy.io import wavfile
from python_speech_features import mfcc
import tensorflow as tf
import os
import re
from tensorflow.python import ops
import time

chars = "abcdefghijklmnopqrstuvwxyz' "
n_inp=26
n_ctx = 9
n_h = 1024
n_chars = len(chars)+1
pb1 = 0.9
pb2 = 0.999
peps = 1e-8
plr = 0.001
epochs=100

/Users/i346047/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/i346047/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def audiofile_to_vector(audio_fname, n_mfcc_features, nctx):
    sampling_rate, raw_w = wavfile.read(audio_fname)
    mfcc_ft = mfcc(raw_w, samplerate=sampling_rate, numcep=n_mfcc_features)
    mfcc_ft = mfcc_ft[::2]
    n_strides = len(mfcc_ft)
    dummy_ctx = np.zeros((nctx, n_mfcc_features), dtype=mfcc_ft.dtype)
    mfcc_ft = np.concatenate((dummy_ctx, mfcc_ft, dummy_ctx))
    w_size = 2*nctx+1
    input_vector = np.lib.stride_tricks.as_strided(mfcc_ft,(n_strides, w_size, n_mfcc_features),
        (mfcc_ft.strides[0], mfcc_ft.strides[0], mfcc_ft.strides[1]),
        writeable=False)
    input_vector = np.reshape(input_vector, [n_strides, -1])
    input_vector = np.copy(input_vector)
    input_vector = (input_vector - np.mean(input_vector))/np.std(input_vector)
    return input_vector

In [3]:
mfcc_features = audiofile_to_vector('../../speech_dset/recordings/train/8_theo_4.wav',26,9)
print(mfcc_features.shape)

(17, 494)


In [4]:
regexp_alphabets = "[^a-zA-Z']+"
cnt=0
def get_label(ch):
    global cnt
    label = cnt
    cnt+=1
    return label
chr2lbl = {c:get_label(c) for c in list(chars)}
lbl2chr = {chr2lbl[c]:c for c in list(chars)}
def get_string2label(strval):
    strval = strval.lower()
    idlist = []
    for c in list(strval):
        if c in chr2lbl:
            idlist.append(chr2lbl[c])
    return np.array(idlist)
def get_label2string(lblarr):
    strval = []
    for idv in lblarr:
        strval.append(lbl2chr[idv])
    return ''.join(strval)
def decoded_val_to_text(decoded_val):
    idxs = decoded_val[0]
    vals = decoded_val[1]
    res = [''] * decoded_val[2][0]
    for i in range(len(idxs)):
        idx = idxs[i][0]
        char = lbl2chr[vals[i]]
        res[idx] = res[idx] + char
    return res
def array2txt(arr_val):
    res = ''
    for i in range(len(arr_val)):
        if arr_val[i] in lbl2chr:
            res += lbl2chr[arr_val[i]]
        else:
            res += ''
    return res.replace('`', ' ')

In [5]:
idlist = get_string2label("This is a test")
print(idlist)
strval = get_label2string(idlist)
print(strval)

[19  7  8 18 27  8 18 27  0 27 19  4 18 19]
this is a test


In [6]:
def get_wav_trans(fpath,X, y):
    files = os.listdir(fpath)
    for fname in files:
        next_path = fpath + "/" + fname
        if os.path.isdir(next_path):
            get_wav_trans(next_path,X,y)
        else:
            if fname.endswith('wav'):
                fname_without_ext = fname.split(".")[0]
                trans_fname = fname_without_ext + ".txt"
                trans_fname_path = fpath + "/" + trans_fname
                if os.path.isfile(trans_fname_path):
                    mfcc_ft = audiofile_to_vector(next_path,n_inp,n_ctx)
                    with open(trans_fname_path,'r') as content:
                        transcript = content.read()
                        transcript = re.sub(regexp_alphabets, ' ', transcript).strip().lower()
                    trans_lbl = get_string2label(transcript)
                    X.append(mfcc_ft)
                    y.append(trans_lbl)

In [7]:
def get_layers(X_batch,seq_len):
    X_batch_shape = tf.shape(X_batch)
    X_batch = tf.transpose(X_batch, [1, 0, 2])
    X_batch = tf.reshape(X_batch, [-1, n_inp + 2*n_inp*n_ctx])
    
    with tf.name_scope('Lyr1'):
        B1 = tf.get_variable(name='B1', shape=[n_h], 
                             initializer=tf.random_normal_initializer(stddev=0.046875))
        H1 = tf.get_variable(name='H1', shape=[n_inp + 2*n_inp*n_ctx, n_h],
                             initializer=tf.contrib.layers.xavier_initializer(uniform=False))
        logits1 = tf.add(tf.matmul(X_batch, H1), B1)
        relu1 = tf.nn.relu(logits1)
        clipped_relu1 = tf.minimum(relu1,20.0)
        Lyr1 = tf.nn.dropout(clipped_relu1, 0.5)
    with tf.name_scope('Lyr2'):
        B2 = tf.get_variable(name='B2', shape=[n_h], 
                             initializer=tf.random_normal_initializer(stddev=0.046875))
        H2 = tf.get_variable(name='H2', shape=[n_h,n_h],
                             initializer=tf.random_normal_initializer(stddev=0.046875))
        logits2 = tf.add(tf.matmul(Lyr1, H2), B2)
        relu2 = tf.nn.relu(logits2)
        clipped_relu2 = tf.minimum(relu2,20.0)
        Lyr2 = tf.nn.dropout(clipped_relu2, 0.5)
    with tf.name_scope('Lyr3'):
        B3 = tf.get_variable(name='B3', shape=[2*n_h], 
                             initializer=tf.random_normal_initializer(stddev=0.046875))
        H3 = tf.get_variable(name='H3', shape=[n_h,2*n_h],
                             initializer=tf.random_normal_initializer(stddev=0.046875))
        logits3 = tf.add(tf.matmul(Lyr2, H3), B3)
        relu3 = tf.nn.relu(logits3)
        clipped_relu3 = tf.minimum(relu3,20.0)
        Lyr3 = tf.nn.dropout(clipped_relu3, 0.5)
    
    with tf.name_scope('RNN_Lyr'):
        fw_c = tf.contrib.rnn.BasicLSTMCell(n_h, forget_bias=1.0, state_is_tuple=True, 
                                            reuse=tf.get_variable_scope().reuse)
        fw_c = tf.contrib.rnn.DropoutWrapper(fw_c, input_keep_prob=0.7, output_keep_prob=0.7,seed=123)
        bw_c = tf.contrib.rnn.BasicLSTMCell(n_h, forget_bias=1.0, state_is_tuple=True, 
                                                    reuse=tf.get_variable_scope().reuse)
        bw_c = tf.contrib.rnn.DropoutWrapper(bw_c,input_keep_prob=0.7, output_keep_prob=0.7,
                                                    seed=123)
        Lyr3 = tf.reshape(Lyr3, [-1, X_batch_shape[0], 2*n_h])
        outs, out_states = tf.nn.bidirectional_dynamic_rnn(cell_fw=fw_c,
                                                                     cell_bw=bw_c,
                                                                     inputs=Lyr3,
                                                                     dtype=tf.float32,
                                                                     time_major=True,
                                                                     sequence_length=seq_len)
        outs = tf.concat(outs, 2)
        outs = tf.reshape(outs, [-1, 2 * n_h])
    
    with tf.name_scope('Lyr4'):
        B4 = tf.get_variable(name='B4', shape=[n_h], 
                             initializer=tf.random_normal_initializer(stddev=0.046875))
        H4 = tf.get_variable(name='H4', shape=[(2 * n_h), n_h],
                             initializer=tf.random_normal_initializer(stddev=0.046875))
        logits4 = tf.add(tf.matmul(outs, H4), B4)
        relu4 = tf.nn.relu(logits4)
        clipped_relu4 = tf.minimum(relu4,20.0)
        Lyr4 = tf.nn.dropout(clipped_relu4, 0.5)
    
    with tf.name_scope('Lyr5'):
        B5 = tf.get_variable(name='B5', shape=[n_chars], 
                             initializer=tf.random_normal_initializer(stddev=0.046875))
        H5 = tf.get_variable(name='H5', shape=[n_h, n_chars],
                             initializer=tf.random_normal_initializer(stddev=0.046875))
        Lyr5 = tf.add(tf.matmul(Lyr4, H5), B5)
        Lyr5 = tf.reshape(Lyr5, [-1, X_batch_shape[0], n_chars])
    
    return Lyr5
    

def get_logits(X_batch,seq_len):
    logits = get_layers(X_batch,seq_len)
    return logits

In [8]:
class Batch:
    def __init__(self):
        self.start_idx = 0
        self.batch_size = 32
        self.audio = []
        self.transcript = []
        get_wav_trans("../../speech_dset/timit/",self.audio,self.transcript)     
    def pad_seq(self,seqs):
        seq_lens = np.asarray([len(st) for st in seqs], dtype=np.int64)
        n_s = len(seqs)
        max_seq_len = np.max(seq_lens)
        s_shape = tuple()
        for s in seqs:
            if len(s) > 0:
                s_shape = np.asarray(s).shape[1:]
                break
        seqs_trc = (np.ones((n_s, max_seq_len) + s_shape) * 0.).astype(np.float32)
        for ix, s in enumerate(seqs):
            if len(s) == 0:
                continue  
            trc = s[:max_seq_len]
            trc = np.asarray(trc, dtype=np.int64)
            if trc.shape[1:] != s_shape:
                raise ValueError("ERROR in truncation shape")
            seqs_trc[ix, :len(trc)] = trc
        return seqs_trc, seq_lens
    def get_sp_tuple(self,seqs):
        ixs = []
        vals = []
        for n, s in enumerate(seqs):
            ixs.extend(zip([n] * len(s), range(len(s))))
            vals.extend(s)
        ixs = np.asarray(ixs, dtype=np.int64)
        vals = np.asarray(vals, dtype=np.int32)
        shape = np.asarray([len(seqs), ixs.max(0)[1] + 1], dtype=np.int64)
        return ixs, vals, shape
    def get_next_batch(self):
        src = self.audio[self.start_idx:self.start_idx+self.batch_size]
        tgt = self.transcript[self.start_idx:self.start_idx+self.batch_size]
        self.start_idx += self.batch_size
        if(self.start_idx>len(self.audio)):
            self.start_idx=0
        src,src_len = self.pad_seq(src)
        sp_lbls = self.get_sp_tuple(tgt)
        return src, src_len, sp_lbls

In [9]:
def get_model():
    input_t = tf.placeholder(tf.float32, [None, None, n_inp + 
                                                (2 * n_inp * n_ctx)], name='inp')
    tgts = tf.sparse_placeholder(tf.int32, name='tgts')
    len_seq = tf.placeholder(tf.int32, [None], name='len_seq')
    logits = get_logits(input_t,tf.to_int64(len_seq))
    return input_t, tgts, len_seq, logits

In [10]:
def get_cost(tgts,logits,len_seq):
    loss_t = ops.ctc_ops.ctc_loss(tgts, logits, len_seq)
    loss_avg = tf.reduce_mean(loss_t)
    return loss_avg

In [11]:
def get_optimizer(logits,len_seq,loss_avg):
    adm_opt = tf.train.AdamOptimizer(learning_rate=plr,beta1=pb1,beta2=pb2,epsilon=peps)
    adm_opt = adm_opt.minimize(loss_avg)
    dec, prob_log = ops.ctc_ops.ctc_beam_search_decoder(logits, len_seq, merge_repeated=False)
    return adm_opt,dec

In [12]:
def get_error_rates(dec,tgts):
    edit_dist = tf.edit_distance(tf.cast(dec[0], tf.int32), tgts)
    error_rate4label = tf.reduce_mean(edit_dist, name='error_rate4label')
    return error_rate4label

In [ ]:
gr = tf.Graph()
with gr.as_default():
    input_t,tgts,len_seq,logits = get_model()
    loss_avg = get_cost(tgts,logits,len_seq)
    adm_opt, dec = get_optimizer(logits,len_seq,loss_avg)
    error_rate = get_error_rates(dec,tgts)
    sess = tf.Session()
    writer = tf.summary.FileWriter('/tmp/models/', graph=sess.graph)
    loss_summary = tf.summary.scalar("loss_avg", loss_avg)
    sum_op = tf.summary.merge_all()
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    for ep in range(epochs):
        train_cost = 0
        label_err_rate = 0
        batch_feeder = Batch()
        n_batches = np.ceil(len(batch_feeder.audio)/batch_feeder.batch_size)
        n_batches = int(n_batches)
        st = time.time()
        for batch in range(n_batches):
            src,len_src,labels_src = batch_feeder.get_next_batch()
            data_dict = {input_t: src, tgts: labels_src,len_seq:len_src}
            batch_cost, _,summ = sess.run([loss_avg, adm_opt,sum_op], data_dict)
            train_cost += batch_cost * batch_feeder.batch_size
            print("Batch cost: {0}, Train cost: {1}".format(batch_cost,train_cost))
            label_err_rate += sess.run(error_rate, feed_dict=data_dict) * batch_feeder.batch_size
            print('Label error: {}'.format(label_err_rate))
            writer.add_summary(summ,ep*batch_feeder.batch_size+batch)
        saver = tf.train.Saver() 
        saver.save(sess, '/tmp/models/speech2txt.ckpt')
        decoded_val = sess.run(dec[0], feed_dict=data_dict)
        d_decoded_val = tf.sparse_tensor_to_dense(decoded_val, default_value=-1).eval(session=sess)
        d_lbl = decoded_val_to_text(labels_src)
        cnt = 0
        cnt_max = 4
        if cnt < cnt_max:
            for actual_val, decoded_val in zip(d_lbl, d_decoded_val):
                d_str = array2txt(decoded_val)
                print('Batch {}'.format(batch))
                print('Actual: {}'.format(actual_val))
                print('Predicted:  {}'.format(d_str))
                cnt += 1
        time_taken = time.time() - st
        log = 'Epoch {}/{}, training_cost: {:.3f}, error_rate: {:.3f}, time: {:.2f} sec'
        print(log.format(ep,epochs,train_cost/len(batch_feeder.audio),
                (label_err_rate/len(batch_feeder.audio)), time_taken))

Batch cost: 440.0248718261719, Train cost: 14080.7958984375
Label error: 30.93742561340332
Batch cost: 1031.940673828125, Train cost: 47102.8974609375
Label error: 62.7877311706543
Batch cost: 590.1786499023438, Train cost: 65988.6142578125
Label error: 89.71720886230469
Batch cost: 375.1274719238281, Train cost: 77992.693359375
Label error: 121.11743545532227
Batch cost: 292.9910888671875, Train cost: 87368.408203125
Label error: 152.5082015991211
Batch 4
Actual: the two artists exchanged autographs
Predicted:   a a a t o a a l s a a t r a i i n s a t sni 
Batch 4
Actual: in simpler terms it amounts to pointing the platform in the proper direction
Predicted:   l r n a s s a a n n ia t a a s n a a s s a a a a a n a a s n s a t a n t 
Batch 4
Actual: do you have the yellow ointment ready
Predicted:   t a n a o t a l t a t a a n tn a a a s s 
Batch 4
Actual: the diagnosis was discouraging however he was not overly worried
Predicted:   l a n e a a s ns s a n rs t a a s a s t a a a a n l t

Batch cost: 212.12335205078125, Train cost: 6787.947265625
Label error: 27.92232894897461
Batch cost: 200.25103759765625, Train cost: 13195.98046875
Label error: 56.30742073059082
Batch cost: 200.69775390625, Train cost: 19618.30859375
Label error: 83.73795700073242
Batch cost: 191.05050659179688, Train cost: 25731.9248046875
Label error: 109.68250274658203
Batch cost: 201.54034423828125, Train cost: 32181.2158203125
Label error: 134.71599578857422
Batch 4
Actual: the two artists exchanged autographs
Predicted:   a a a a a a 
Batch 4
Actual: in simpler terms it amounts to pointing the platform in the proper direction
Predicted:   a a a a a a a a e e 
Batch 4
Actual: do you have the yellow ointment ready
Predicted:   a a a a s 
Batch 4
Actual: the diagnosis was discouraging however he was not overly worried
Predicted:   a a a a a a a a a a a a a 
Batch 4
Actual: don't ask me to carry an oily rag like that
Predicted:   a a a a a a 
Batch 4
Actual: she had your dark suit in greasy wash wa

Batch cost: 173.17974853515625, Train cost: 5541.751953125
Label error: 25.99690055847168
Batch cost: 171.01173400878906, Train cost: 11014.12744140625
Label error: 50.681705474853516
Batch cost: 168.1267547607422, Train cost: 16394.18359375
Label error: 74.50191879272461
Batch cost: 169.2356719970703, Train cost: 21809.72509765625
Label error: 99.12186431884766
Batch cost: 170.70291137695312, Train cost: 27272.21826171875
Label error: 124.25790214538574
Batch 4
Actual: the two artists exchanged autographs
Predicted:   e e a a a ate tae
Batch 4
Actual: in simpler terms it amounts to pointing the platform in the proper direction
Predicted:  ehte t a a a a a a a a 
Batch 4
Actual: do you have the yellow ointment ready
Predicted:   a a a a a a 
Batch 4
Actual: the diagnosis was discouraging however he was not overly worried
Predicted:   e e a a e e t e a atetaeae
Batch 4
Actual: don't ask me to carry an oily rag like that
Predicted:  eta e a 
Batch 4
Actual: she had your dark suit in grea

Batch cost: 162.9083251953125, Train cost: 5213.06640625
Label error: 24.134414672851562
Batch cost: 163.58724975585938, Train cost: 10447.8583984375
Label error: 47.52470016479492
Batch cost: 161.64825439453125, Train cost: 15620.6025390625
Label error: 71.44134140014648
Batch cost: 155.58824157714844, Train cost: 20599.42626953125
Label error: 95.57803344726562
Batch cost: 162.81639099121094, Train cost: 25809.55078125
Label error: 119.47598266601562
Batch 4
Actual: the two artists exchanged autographs
Predicted:   e e e a e e e a a e 
Batch 4
Actual: in simpler terms it amounts to pointing the platform in the proper direction
Predicted:   e e e e e e e e e e e e e a a 
Batch 4
Actual: do you have the yellow ointment ready
Predicted:   e e e o a e e e 
Batch 4
Actual: the diagnosis was discouraging however he was not overly worried
Predicted:   e e a a e e e e a e e e e e e a a
Batch 4
Actual: don't ask me to carry an oily rag like that
Predicted:   e e e e a e 
Batch 4
Actual: she h

Batch cost: 154.9497833251953, Train cost: 4958.39306640625
Label error: 25.139310836791992
Batch cost: 152.81500244140625, Train cost: 9848.47314453125
Label error: 50.49465751647949
Batch cost: 156.73178100585938, Train cost: 14863.89013671875
Label error: 76.20839500427246
Batch cost: 150.35726928710938, Train cost: 19675.32275390625
Label error: 101.47273254394531
Batch cost: 157.46990966796875, Train cost: 24714.35986328125
Label error: 126.33822059631348
Batch 4
Actual: the two artists exchanged autographs
Predicted:  e    e e     ea
Batch 4
Actual: in simpler terms it amounts to pointing the platform in the proper direction
Predicted:   a a a e  t    a  e       a  e
Batch 4
Actual: do you have the yellow ointment ready
Predicted:     e      ea
Batch 4
Actual: the diagnosis was discouraging however he was not overly worried
Predicted:   ae e   e    e   es
Batch 4
Actual: don't ask me to carry an oily rag like that
Predicted:   a  e      e
Batch 4
Actual: she had your dark suit in

...

Batch cost: 115.48899841308594, Train cost: 3695.64794921875
Label error: 20.856565475463867
Batch cost: 106.5296401977539, Train cost: 7104.596435546875
Label error: 40.057273864746094
Batch cost: 112.21418762207031, Train cost: 10695.450439453125
Label error: 60.50319290161133
Batch cost: 111.35103607177734, Train cost: 14258.68359375
Label error: 80.95959854125977
Batch cost: 119.31396484375, Train cost: 18076.73046875
Label error: 101.99831962585449
Batch 4
Actual: the two artists exchanged autographs
Predicted:  the ars i aes
Batch 4
Actual: in simpler terms it amounts to pointing the platform in the proper direction
Predicted:  is oe te ae t s toe a ae oi
Batch 4
Actual: do you have the yellow ointment ready
Predicted:  oe o re oras
Batch 4
Actual: the diagnosis was discouraging however he was not overly worried
Predicted:  th oe as ar tn aet ar orra
Batch 4
Actual: don't ask me to carry an oily rag like that
Predicted:  dont sk me o rry a oky rag like that
Batch 4
Actual: she ha

Batch cost: 114.05288696289062, Train cost: 3649.6923828125
Label error: 20.39466094970703
Batch cost: 106.65473937988281, Train cost: 7062.64404296875
Label error: 39.579795837402344
Batch cost: 114.54888916015625, Train cost: 10728.20849609375
Label error: 59.73141860961914
Batch cost: 112.308349609375, Train cost: 14322.07568359375
Label error: 80.01721382141113
Batch cost: 117.00566864013672, Train cost: 18066.257080078125
Label error: 101.36240577697754
Batch 4
Actual: the two artists exchanged autographs
Predicted:  the ais te aes
Batch 4
Actual: in simpler terms it amounts to pointing the platform in the proper direction
Predicted:  is oe  on n ae on on te an
Batch 4
Actual: do you have the yellow ointment ready
Predicted:  da h a r one
Batch 4
Actual: the diagnosis was discouraging however he was not overly worried
Predicted:  the o as tiey e ase oy i
Batch 4
Actual: don't ask me to carry an oily rag like that
Predicted:  dn't ask me a ary an oily rag like hat
Batch 4
Actual: s

Batch cost: 110.57310485839844, Train cost: 3538.33935546875
Label error: 19.61601448059082
Batch cost: 103.6232681274414, Train cost: 6854.283935546875
Label error: 38.29738426208496
Batch cost: 108.69564819335938, Train cost: 10332.544677734375
Label error: 58.5506477355957
Batch cost: 106.4952392578125, Train cost: 13740.392333984375
Label error: 78.85557174682617
Batch cost: 114.49174499511719, Train cost: 17404.128173828125
Label error: 99.56673812866211
Batch 4
Actual: the two artists exchanged autographs
Predicted:  the oie te oras
Batch 4
Actual: in simpler terms it amounts to pointing the platform in the proper direction
Predicted:  t ue a a o o  wo o ai
Batch 4
Actual: do you have the yellow ointment ready
Predicted:  o he as  e oay
Batch 4
Actual: the diagnosis was discouraging however he was not overly worried
Predicted:  the oe oae ore ae a eoe oea
Batch 4
Actual: don't ask me to carry an oily rag like that
Predicted:  do't ak me o ar an a oily rag like hat
Batch 4
Actual:

Batch cost: 111.06185150146484, Train cost: 3553.979248046875
Label error: 19.867345809936523
Batch cost: 98.75746154785156, Train cost: 6714.218017578125
Label error: 38.75369071960449
Batch cost: 106.38020324707031, Train cost: 10118.384521484375
Label error: 58.685434341430664
Batch cost: 107.29460144042969, Train cost: 13551.811767578125
Label error: 78.26999855041504
Batch cost: 111.69200134277344, Train cost: 17125.955810546875
Label error: 98.1200122833252
Batch 4
Actual: the two artists exchanged autographs
Predicted:  the es tid ais
Batch 4
Actual: in simpler terms it amounts to pointing the platform in the proper direction
Predicted:  aes tue te ain  s oe oe an
Batch 4
Actual: do you have the yellow ointment ready
Predicted:  a a ie a  aer hread
Batch 4
Actual: the diagnosis was discouraging however he was not overly worried
Predicted:  the ao n e ry oe an are aay orar
Batch 4
Actual: don't ask me to carry an oily rag like that
Predicted:  don't ask me  cary an oily rag like 

Batch cost: 107.51701354980469, Train cost: 3440.54443359375
Label error: 19.371578216552734
Batch cost: 97.26266479492188, Train cost: 6552.94970703125
Label error: 37.33325004577637
Batch cost: 106.36431884765625, Train cost: 9956.60791015625
Label error: 56.76771926879883
